In [2]:
!pip install Kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 19.5 MB/s eta 0:00:00
  Created wheel for Kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105137 sha256=a82829efbb0782cfac0825b42868f5c52f0d508276961bb1ebfa596b9d34106c
  Stored in directory: /root/.cache/pip/wheels/d7/54/06/8a8f40cb39536605feb9acaacd0237a95eba39e5065e6392f4
Successfully built Kaggle


In [7]:
import os
import json
import requests
from tqdm import tqdm

# ตั้งค่า Kaggle API key
kaggle_json = {
    "username": "project2youcompany",
    "key": "7310f356c2d2587f78bc7bdb6bdaae21"
}

# สร้างไฟล์ kaggle.json
kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)
with open(os.path.join(kaggle_dir, 'kaggle.json'), 'w') as f:
    json.dump(kaggle_json, f)

# ตั้งค่า permission ให้กับไฟล์ kaggle.json
os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)

# กำหนด directory สำหรับเก็บ data sets
dataset_dir = 'kaggle_datasets'
os.makedirs(dataset_dir, exist_ok=True)

# ชื่อ dataset ที่ต้องการโหลด
dataset = 'adityajn105/flickr8k'

# ดึง URL สำหรับดาวน์โหลดไฟล์
kaggle_api_url = f"https://www.kaggle.com/api/v1/datasets/download/{dataset}"

headers = {
    "Authorization": f"Bearer {kaggle_json['key']}"
}

response = requests.get(kaggle_api_url, headers=headers, stream=True)
if response.status_code != 200:
    raise Exception(f"Error downloading dataset: {response.status_code}")

# ดึงชื่อไฟล์จาก URL
zip_filename = os.path.join(dataset_dir, f"{dataset.split('/')[-1]}.zip")

# ดาวน์โหลดไฟล์พร้อมแสดงแถบความคืบหน้า
total_size_in_bytes = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kibibyte
progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

with open(zip_filename, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()

# ตรวจสอบว่าดาวน์โหลดไฟล์ครบถ้วนหรือไม่
if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
    raise Exception("ERROR, something went wrong with the download")

# แตกไฟล์ zip ที่โหลดมา
import zipfile

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

os.remove(zip_filename)  # ลบไฟล์ zip หลังจากแตกไฟล์แล้ว

print(f'Dataset {dataset} has been downloaded and extracted to {dataset_dir}')


100%|██████████| 1.11G/1.11G [01:00<00:00, 18.4MiB/s] 


Dataset adityajn105/flickr8k has been downloaded and extracted to kaggle_datasets
